In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# importing libraries
import pandas as pd
import numpy as np
import shap
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.feature_selection import chi2
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use("fivethirtyeight")
sns.set_style("darkgrid")
import warnings
warnings.filterwarnings(action="ignore")

In [30]:
#Check the data
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")
df.head(10)

### **We don't know what is the meaning of each feature.**
### Classes
    - Fraud = 1
    - Not Fraud = 0

In [31]:
# Data Shape (number of rows and columns)
print(f"Dataset has {df.shape[1]} columns and {df.shape[0]} rows")

In [6]:
df = df.astype("int32")

In [11]:
df.info()

In [12]:
# Percentage of null values
(df.isna().sum())/len(df)

In [13]:
df.describe()

In [14]:
plt.figure(figsize=(15,14))
sns.heatmap(df.corr(), annot=True, fmt=".1f",cmap="RdBu")

In [15]:
plt.figure(figsize=(8,5))
sns.distplot(df["Time"],kde=True, bins=50,color="red")

In [16]:
sns.boxplot(df["Amount"],orient="h")

In [17]:
sns.boxplot(df["Amount"],orient="h")

In [18]:
sns.countplot(df["Class"],palette="Set1")
plt.title("Countplot of Target Variable")

In [19]:
rbst = RobustScaler()
df['Amount'] = rbst.fit_transform(df['Amount'].values.reshape(-1,1))
df['Time'] = rbst.fit_transform(df['Time'].values.reshape(-1,1))

In [20]:
X = df.drop("Class",axis=1)
y = df.Class

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.25,random_state=24)

In [21]:
smote = SMOTE(random_state=24)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

In [22]:
xgb = XGBClassifier()
xgb.fit(X_smote, y_smote)

In [23]:
y_pred3 = xgb.predict(X_test)
report3 = classification_report(y_pred3, y_test)
print(report3)
print(f"Accuracy: {accuracy_score(y_pred3, y_test)}")
print(f"ROC AUC Score for XGBoost Classifier: {roc_auc_score(y_pred3, y_test)}")

In [24]:
cat_classifier = CatBoostClassifier(iterations=250,verbose=False)
cat_classifier.fit(X_smote, y_smote)

In [25]:
y_pred4 = cat_classifier.predict(X_test)
report4 = classification_report(y_pred4, y_test)
print(report4)
print(f"Accuracy: {accuracy_score(y_pred4, y_test)}")
print(f"ROC AUC Score for Cat Boost Classifier: {roc_auc_score(y_pred4, y_test)}")

In [27]:
rf = RandomForestClassifier()
rf.fit(X_smote, y_smote)

In [28]:
y_pred2 = rf.predict(X_test)
report2 = classification_report(y_pred2, y_test)
print(report2)
print(f"Accuracy: {accuracy_score(y_pred2, y_test)}")
print(f"ROC AUC Score for Random Forest Classifier: {roc_auc_score(y_pred2, y_test)}")

In [29]:
perm = PermutationImportance(rf,random_state=24).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())